In [74]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [75]:
#read initial csv into a dataframe
csv_file = 'Main Avocado Tree (Resources)/avocado-updated-2020.csv'
avocado_df = pd.read_csv(csv_file)
avocado_df

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,geography
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.00,conventional,2015,Albany
1,2015-01-04,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.00,organic,2015,Albany
2,2015-01-04,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.00,conventional,2015,Atlanta
3,2015-01-04,1.76,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.00,organic,2015,Atlanta
4,2015-01-04,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.00,conventional,2015,Baltimore/Washington
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33040,2020-11-29,1.47,1583056.27,67544.48,97996.46,2617.17,1414878.10,906711.52,480191.83,27974.75,organic,2020,Total U.S.
33041,2020-11-29,0.91,5811114.22,1352877.53,589061.83,19741.90,3790665.29,2197611.02,1531530.14,61524.13,conventional,2020,West
33042,2020-11-29,1.48,289961.27,13273.75,19341.09,636.51,256709.92,122606.21,134103.71,0.00,organic,2020,West
33043,2020-11-29,0.67,822818.75,234688.01,80205.15,10543.63,497381.96,285764.11,210808.02,809.83,conventional,2020,West Tex/New Mexico


In [76]:
#read income csv into a dataframe
csv_income = 'Main Avocado Tree (Resources)/income per metropolitan area.csv'
income_df = pd.read_csv(csv_income)
income_df.head()

,Geography,State,Year,Per capita personal income
0,Abilene,TX,2017,40876
1,Akron,OH,2017,47522
2,Albany,GA,2017,36780
3,Albany-Lebanon,OR,2017,41169
4,Albany-Schenectady-Troy,NY,2017,56487


In [77]:
#Rename columns to match those in Avocado
rename_income_df = income_df.rename(columns={"Geography": "geography", "Year":"year"})
rename_income_df.head()

,geography,State,year,Per capita personal income
0,Abilene,TX,2017,40876
1,Akron,OH,2017,47522
2,Albany,GA,2017,36780
3,Albany-Lebanon,OR,2017,41169
4,Albany-Schenectady-Troy,NY,2017,56487


In [78]:
#Merge the two dataframes together on Geography and year
merged_df = pd.merge(avocado_df,rename_income_df, on=['geography', 'year'], how='inner')
merged_df

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,geography,State,Per capita personal income
0,2017-01-01,1.47,129948.23,4845.77,117027.41,200.36,7874.69,7866.86,7.83,0.00,conventional,2017,Albany,GA,36780
1,2017-01-01,1.87,1376.70,71.65,192.63,0.00,1112.42,1112.42,0.00,0.00,organic,2017,Albany,GA,36780
2,2017-01-08,1.55,91728.18,3355.47,75641.23,56.91,12674.57,12606.67,67.90,0.00,conventional,2017,Albany,GA,36780
3,2017-01-08,1.94,2229.52,63.46,478.31,0.00,1687.75,1687.75,0.00,0.00,organic,2017,Albany,GA,36780
4,2017-01-15,1.55,88526.26,3327.65,71956.77,607.03,12634.81,12574.72,60.09,0.00,conventional,2017,Albany,GA,36780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,2019-12-15,1.34,3895.70,253.86,130.50,0.00,3511.34,1161.84,2349.50,0.00,organic,2019,Syracuse,NY,52051
3076,2019-12-22,1.09,75058.95,3277.09,42043.81,50.00,29688.05,9821.50,18648.77,1217.78,conventional,2019,Syracuse,NY,52051
3077,2019-12-22,1.58,2610.75,124.88,72.44,0.00,2413.43,1181.44,1231.99,0.00,organic,2019,Syracuse,NY,52051
3078,2019-12-29,1.12,67860.40,2800.47,39432.09,51.00,25576.84,6601.50,17943.11,1032.23,conventional,2019,Syracuse,NY,52051


In [79]:
#Changing Date to ONLY reflect the month
split_cols = merged_df['date'].str.split('-',expand=True)
split_cols.columns = [f'Split-{i}' for i in range(3)]
new_merge = merged_df.join(split_cols)
new_merge

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,geography,State,Per capita personal income,Split-0,Split-1,Split-2
0,2017-01-01,1.47,129948.23,4845.77,117027.41,200.36,7874.69,7866.86,7.83,0.00,conventional,2017,Albany,GA,36780,2017,01,01
1,2017-01-01,1.87,1376.70,71.65,192.63,0.00,1112.42,1112.42,0.00,0.00,organic,2017,Albany,GA,36780,2017,01,01
2,2017-01-08,1.55,91728.18,3355.47,75641.23,56.91,12674.57,12606.67,67.90,0.00,conventional,2017,Albany,GA,36780,2017,01,08
3,2017-01-08,1.94,2229.52,63.46,478.31,0.00,1687.75,1687.75,0.00,0.00,organic,2017,Albany,GA,36780,2017,01,08
4,2017-01-15,1.55,88526.26,3327.65,71956.77,607.03,12634.81,12574.72,60.09,0.00,conventional,2017,Albany,GA,36780,2017,01,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,2019-12-15,1.34,3895.70,253.86,130.50,0.00,3511.34,1161.84,2349.50,0.00,organic,2019,Syracuse,NY,52051,2019,12,15
3076,2019-12-22,1.09,75058.95,3277.09,42043.81,50.00,29688.05,9821.50,18648.77,1217.78,conventional,2019,Syracuse,NY,52051,2019,12,22
3077,2019-12-22,1.58,2610.75,124.88,72.44,0.00,2413.43,1181.44,1231.99,0.00,organic,2019,Syracuse,NY,52051,2019,12,22
3078,2019-12-29,1.12,67860.40,2800.47,39432.09,51.00,25576.84,6601.50,17943.11,1032.23,conventional,2019,Syracuse,NY,52051,2019,12,29


In [80]:
#make it pretty
renamed_merge = new_merge.rename(columns={'Split-1':'month', 'geography':'city'})
clean_merge = renamed_merge[['month', 'year', 'city', 'average_price', 'total_volume','Per capita personal income', '4046', '4225', '4770', 'type']]
clean_merge.head()

,month,year,city,average_price,total_volume,Per capita personal income,4046,4225,4770,type
0,01,2017,Albany,1.47,129948.23,36780,4845.77,117027.41,200.36,conventional
1,01,2017,Albany,1.87,1376.70,36780,71.65,192.63,0.00,organic
2,01,2017,Albany,1.55,91728.18,36780,3355.47,75641.23,56.91,conventional
3,01,2017,Albany,1.94,2229.52,36780,63.46,478.31,0.00,organic
4,01,2017,Albany,1.55,88526.26,36780,3327.65,71956.77,607.03,conventional


In [81]:
# Question 1: Which cities love avocados most?

#Task 1: popularity of avocado, visual heatmap, 2015 to 2020, bar graph of volume per area 

In [82]:
#Question 2: Do states with higher income per capita buy more avocados? (income per capita from 2017-2019)

#Task 2: line graph (x-values states, y-values income per capita and avocado revenue) 

In [94]:
#Question 3: Which type of avocado is the biggest source of revenue for suppliers? Erica
#Include Day (Split-2) column
revenue_df = renamed_merge.rename(columns={'Split-2':'day'})
revenue_df = revenue_df[['day','month', 'year', 'city', 'average_price', 'total_volume','Per capita personal income', '4046', '4225', '4770', 'type']]
cities = revenue_df['city'].unique()
cities
#Task 3: Five pie charts by type (value = revenue) for each year on one png 

array(['Albany', 'Columbus', 'Jacksonville', 'Pittsburgh', 'Roanoke',
       'St. Louis', 'Syracuse'], dtype=object)

In [84]:
#Question 4: Is there a time of year that sees a spike in revenue? Erica

#Task 4: bar chart (x-values months, y-values avocado revenue)

In [85]:
#Question 5: How has the average avocado price changed over time?  What is the predicted avocado price for 2021 based on our data?  Bitty

#Task 5: scatter plot with regression to predict 2021 average price 

In [86]:
#Question 6: Do different types of avocados have different price trends? Arianne

#Task 6: facet (multiple lines on one) line graph (x-values = year, y-values = average price) for each type

In [87]:
#look at volume for 4046
plot_4046 = [['4046','year','average_price']]


In [88]:
#look at volume for 4225
plot_4225= [['4225', 'year', 'average_price']]

In [89]:
#Look at Volume for 4770
plot_4770 = [['4770', 'year', 'average_price']]
